# Particle segmentation App

In [1]:
import os , glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
import nanoparticles as nano
import skimage

In [2]:
path_widget = widgets.Select(options = ['None','...']+os.listdir(), value = 'None')

scale_widget = widgets.FloatText(step = 0.1, value=0.5, description='Scale:',disabled=False)

match_widget = widgets.FloatSlider(description = 'Quality',min = 0, max = 1, 
                                   step = 0.1, value = 0.5, continuous_update = False)

minrad_widget = widgets.IntSlider(description = 'Min radius',min = 0, max = 100, 
                                   step = 10, value = 30, continuous_update = False)

maxrad_widget = widgets.IntSlider(description = 'Max radius',min = 0, max = 100, 
                                   step = 10, value = 50, continuous_update = False)

do_widget = widgets.Dropdown(
    options=['No', 'Single', 'All'],
    value='No',
    description='Running:',
    disabled=False,
)

In [3]:
def analyze_particles(path_to_file, min_rad, max_rad, scale, match_quality = 0.2, do_analysis = False):
    """Main function analyzing particles size in EM images
    
    Parameters
    ----------
    path_to_file : str
        path to folder containing tif files
    minrad: int
        minimum radius to consider
    maxrad: int
        maximum radius to consider
    scale: float
        image scale (in nanometer per pixel)
    match_quality: float
        threshold on matching quality. 0 means all local maxima are selected
    do_analysis: bool
        Run analysis or not
    
    Returns
    -------
    all_radii: list
        list of all measured radii in all images
    
    """    
    
    if path_to_file == 'None':
        None
    elif path_to_file =='...':
        newpath = os.path.dirname(os.getcwd())
        os.chdir(newpath)
        if len(os.listdir(newpath))==0:
            path_widget.value = 'None'
            path_widget.options = ['None', '...']
        else:
            #os.chdir(newpath)
            path_widget.options = ['None','...']+os.listdir()
            path_widget.value = 'None'#os.listdir(newpath)[0]

    else:
        newpath = os.path.abspath(os.curdir)+'/'+path_to_file
        if os.path.isdir(newpath):
            os.chdir(newpath)
            if len(os.listdir(newpath))==0:
                path_widget.options = ['None', '...']
                path_widget.value = 'None'
            else:
                #os.chdir(newpath)
                path_widget.options = ['None','...']+os.listdir()
                path_widget.value = 'None'#os.listdir(newpath)[0]
                
    
    path_to_file = os.path.abspath(os.curdir)
    
    
    image = []
    circles = []
    all_radii = []
    if do_analysis != 'No':
        
        tif_files = glob.glob(path_to_file+'/*tif')
        if do_analysis == 'Single':
            tif_files = tif_files[0:1]

        all_radii = []
        for tif in tif_files:
            image = skimage.io.imread(tif)
            if len(image.shape)==3:
                image = image[:,:,1]+0.001
            im_filt = nano.init_filtering(image, np.arange(min_rad, max_rad,10))
            radii, circles, intensities = nano.muli_radius_fitting(image, im_filt, min_rad, max_rad, match_quality)

            fig, ax = plt.subplots(1,2,figsize=(15,7))
            ax[0].imshow(image)

            if len(circles) > 0:
                for x in circles:
                    plot_circ = plt.Circle((x[1], x[0]), x[2], color='r', faceColor = [1,1,1,0])
                    ax[0].add_artist(plot_circ)
                ax[0].set_axis_off()

            if len(radii) > 0:
                ax[1].hist(np.array(radii)*scale, bins = np.linspace(np.min(radii),np.max(radii),10)*scale)
                ax[1].set_xlabel('Radius [nm]')
            plt.show()

            #plot_detection(image, circles, radii, scale)
            pd_temp = pd.DataFrame({'radii': radii,'intensities': intensities, 'filename': os.path.basename(tif)})
            all_radii.append(pd_temp)
        all_radii = pd.concat(all_radii)
    return image, circles, all_radii

In [4]:
out = widgets.interactive_output(analyze_particles,
                                 {'path_to_file': path_widget,
                                  'match_quality': match_widget,
                                 'min_rad': minrad_widget,
                                 'max_rad': maxrad_widget,
                                 'scale': scale_widget,
                                 'do_analysis': do_widget})
ui = widgets.VBox([path_widget, match_widget, minrad_widget, maxrad_widget,scale_widget, do_widget])

In [6]:
display(ui, out)

Output()